# Carga de datos 


## Librerias 

In [62]:

# Importación de las funciones creadas en nuestro archivo de soporte
# -----------------------------------------------------------------------
import sys
sys.path.append("..")

# from src import soporte_carga as sc
from src import soporte_queries_creacion as sqc


# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd
# Para trabajar con postgresql
# -----------------------------------------------------------------------
import psycopg2

# Para trabajar gestionar los nulos
# -----------------------------------------------------------------------
import numpy as np

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

from psycopg2 import errorcodes, OperationalError

In [83]:
def conectar(ddbb):
    try:
        connection = psycopg2.connect(
            database = ddbb,
            user = "postgres",
            password = "admin",
            host = "localhost",
            port = "5432"
        )
    except OperationalError as e:
        if e.pgcode == errorcodes.INVALID_PASSWORD:
            print("La constraseña es errónea.")
        elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
            print("Error de conexión")
        else:
            print(f'Error:{e}')
    return connection

def query_fetch(connection, query_text):
    cursor = connection.cursor()
    cursor.execute(query_text)
    result = cursor.fetchall()
    cursor.close()
    connection.close()
    return result

def query_commit(connection, query_text, *valores):
    cursor = connection.cursor()
    cursor.execute(query_text, *valores)
    connection.commit()
    cursor.close()
    connection.close()
    return print("Done!")

def query_commit_many(connection, query_text, valores):
    cursor = connection.cursor()
    cursor.executemany(query_text, valores)
    connection.commit()
    cursor.close()
    connection.close()
    return print("Done!")

## Creacionde base de datos

In [64]:
def establecer_conn(database_name, postgres_pass, usuario, host="localhost"):
    """
    Establece una conexión a una base de datos de PostgreSQL.

    Params:
        - database_name (str): El nombre de la base de datos a la que conectarse.
        - postgres_pass (str): La contraseña del usuario de PostgreSQL.
        - usuario (str): El nombre del usuario de PostgreSQL.
        - host (str, opcional): La dirección del servidor PostgreSQL. Por defecto es "localhost".

    Returns:
        psycopg2.extensions.connection: La conexión establecida a la base de datos PostgreSQL.

    """

    # Crear la conexión a la base de datos PostgreSQL
    conn = psycopg2.connect(
        host=host,
        user=usuario,
        password=postgres_pass,
        database=database_name
    )

    # Establecer la conexión en modo autocommit
    conn.autocommit = True # No hace necesario el uso del commit al final de cada sentencia de insert, delete, etc.
    
    return conn



# conexion a postgres
conn = establecer_conn("postgres", "admin", "postgres") # Nos conectamos a la base de datos de postgres por defecto para poder crear la nueva base de datos

# creamos un cursor con la conexion que acabamos de crear
cur = conn.cursor()


In [65]:
cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", ("sanidad42",)) 

In [66]:

# Almacenamos en una variable el resultado del fetchone. Si existe tendrá una fila sino será None
bbdd_existe = cur.fetchone() # recordamos que fetchone nos devuelve unico valor  
type(bbdd_existe)

tuple

In [67]:
# Si bbdd_existe es None, creamos la base de datos
if not bbdd_existe:
    cur.execute(f'CREATE DATABASE {"sanidad42"};')
    print(f'Base de datos {"sanidad42"} creada con éxito')
else:
    print(f"La base de datos ya existe")
    

La base de datos ya existe


In [68]:
# Cerramos el cursor y la conexion
cur.close()
conn.close()

In [69]:
def crear_db(database_name):
    # conexion a postgres
    conn = establecer_conn("postgres", "admin", "postgres") # Nos conectamos a la base de datos de postgres por defecto para poder crear la nueva base de datos
    
    # creamos un cursor con la conexion que acabamos de crear
    cur = conn.cursor()
    
    cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (database_name,))
    
    # Almacenamos en una variable el resultado del fetchone. Si existe tendrá una fila sino será None
    bbdd_existe = cur.fetchone()
    
    # Si bbdd_existe es None, creamos la base de datos
    if not bbdd_existe:
        cur.execute(f"CREATE DATABASE {database_name};")
        print(f"Base de datos {database_name} creada con éxito")
    else:
        print(f"La base de datos ya existe")
        
    # Cerramos el cursor y la conexion
    cur.close()
    conn.close()


## Creacionde base de Tablas


In [70]:
conexion = establecer_conn("sanidad42", "admin", "postgres")

# creamos cada una de las tablas de la base de datos.
conexion.cursor().execute(sqc.query_creation_hospitales)
conexion.cursor().execute(sqc.query_creation_tipo_hosp)
conexion.cursor().execute(sqc.query_creation_gastos)
conexion.cursor().execute(sqc.query_creation_ingresos)

In [71]:
gastos = pd.read_csv('..\datos\gastos.csv')
ingresos = pd.read_csv('..\datos\ingresos.csv', index_col=0)

In [72]:
len(gastos["ncodi"].unique())

857

In [73]:
len(ingresos["ncodi"].unique())

2424

In [74]:
ingresos["ncodi"] = ingresos["ncodi"].str.replace(',', '.').astype(float)

In [75]:
ingresos[ingresos["ncodi"].isna()]

,particulares,aseguradoras,aseguradoras_enfermedad,aseguradoreas_trafico,mutuas,tipo,año,ncodi
18510,1011.0,44506.0,44506.0,0.0,0.0,Hospital,2017,NaN
19288,1906.0,13026.0,13026.0,0.0,0.0,consulExter,2017,NaN
20066,3133.0,37131.0,37131.0,0.0,0.0,CMA,2017,NaN
20844,0.0,7622.0,7622.0,0.0,0.0,hospDia,2017,NaN
21622,5885.0,119212.0,118687.0,525.0,1188.0,Urgencia,2017,NaN
22400,0.0,0.0,0.0,0.0,0.0,hospDom,2017,NaN
28085,404302.0,0.0,0.0,0.0,0.0,Hospital,2019,NaN
28862,0.0,0.0,0.0,0.0,0.0,consulExter,2019,NaN
29639,0.0,0.0,0.0,0.0,0.0,CMA,2019,NaN
30416,0.0,0.0,0.0,0.0,0.0,hospDia,2019,NaN


In [76]:
gastos[gastos["ncodi"].isna()]

,año,ncodi,totalcompra,producfarma,materialsani,implantes,restomateriasani,servcontratado,trabajocontratado,xrestocompras,variaexistencias,servexteriores,sumistro,xrestoserviexter,gastopersonal,sueldos,indemnizacion,segsocempresa,otrgassocial,dotaamortizacion,perdidadeterioro,xrestogasto,totcompragasto
1251,2019,NaN,195702,96335.0,35161,0.0,35161.0,8086.0,32820.0,23300.0,3200.0,918446,29685.0,888761.0,2327709,1829339,33779.0,460158,4433.0,29488.0,0.0,2396.0,3476941
3399,2017,NaN,480139,53011.0,121076,0.0,121076.0,89350.0,0.0,216702.0,-81828.0,466118,93226.0,372892.0,470110,366438,2929.0,100743,0.0,234984.0,0.0,190.0,1569713


In [77]:
gastos.drop(index=gastos[gastos["ncodi"].isna()].index, inplace=True)
ingresos.drop(index=ingresos[ingresos["ncodi"].isna()].index, inplace=True)

gastos["ncodi"] = gastos["ncodi"].astype(int)
ingresos["ncodi"] = ingresos["ncodi"].astype(int)

In [78]:
ingresos["ncodi"].unique()[:10]

array([990, 680, 411, 241, 558, 167, 248, 224, 591, 586])

In [79]:
gastos["ncodi"].unique()[:10]

array([ 113,  619,  163,  131,  375, 1090,  764,   14,  657,  362])

In [80]:
result = np.isin(gastos["ncodi"].unique(), ingresos["ncodi"].unique()).all()
print("Están todos los hospitales de los gastos en los ingresos?:", (result)) 

Están todos los hospitales de los gastos en los ingresos?: True


In [95]:
hospitales = ingresos["ncodi"].unique()
lista_tuplas = [tuple([int(i)]) for i in hospitales]
print(lista_tuplas)

[(990,), (680,), (411,), (241,), (558,), (167,), (248,), (224,), (591,), (586,), (453,), (113,), (619,), (163,), (131,), (375,), (1090,), (764,), (14,), (657,), (362,), (989,), (145,), (511,), (530,), (6,), (161,), (623,), (489,), (875,), (940,), (946,), (1023,), (1054,), (1057,), (1031,), (1032,), (1064,), (226,), (687,), (745,), (109,), (808,), (971,), (128,), (919,), (725,), (3,), (386,), (306,), (15,), (264,), (867,), (910,), (1053,), (1055,), (1081,), (1099,), (529,), (345,), (515,), (174,), (523,), (697,), (239,), (287,), (377,), (692,), (335,), (435,), (423,), (351,), (810,), (830,), (892,), (921,), (991,), (992,), (993,), (1070,), (1095,), (1094,), (473,), (663,), (552,), (772,), (568,), (398,), (394,), (614,), (235,), (616,), (267,), (565,), (220,), (278,), (107,), (141,), (711,), (204,), (579,), (348,), (766,), (761,), (728,), (280,), (222,), (276,), (11,), (302,), (682,), (279,), (282,), (281,), (183,), (610,), (116,), (115,), (114,), (126,), (119,), (770,), (583,), (136,), 

In [96]:
query_hospitales = '''INSERT INTO hospitales(ncodi)
                values(%s)'''

conexion = conectar(ddbb = 'sanidad42')
query_commit_many(connection = conexion, query_text=query_hospitales, valores = lista_tuplas)

Done!


In [103]:
tipo = ingresos["tipo"].unique()
lista_tuplas_tipo = [tuple([t]) for t in tipo ]

query_tipos = '''INSERT INTO tipo_hospitalizacion(nombre)
                values(%s)'''

conexion = conectar(ddbb = 'sanidad42')
query_commit_many(connection = conexion, query_text=query_tipos, valores = lista_tuplas_tipo)

Done!


In [112]:
tipo_dict = dict()
for k, i in enumerate(tipo, start=1):
    tipo_dict[i] = k

In [113]:
tipo_dict

{'Hospital': 1,
 'consulExter': 2,
 'CMA': 3,
 'hospDia': 4,
 'Urgencia': 5,
 'hospDom': 6}

In [115]:
ingresos["tipo"] = ingresos["tipo"].map(tipo_dict)

In [120]:
lista_tuplas_ingresos = [tuple(dato) for dato in ingresos.values]

In [123]:
query_ingresos = '''
                    INSERT INTO ingresos(particulares,aseguradoras,aseguradoras_enfermedad,aseguradoras_trafico,mutuas,tipo_id,anio,ncodi)
                        values(%s,%s,%s,%s,%s,%s,%s,%s)
'''

conexion = conectar(ddbb = 'sanidad42')
query_commit_many(connection = conexion, query_text=query_ingresos, valores = lista_tuplas_ingresos)


Done!


In [148]:
lista_tuplas_gastos = [tuple(datos) for datos in gastos.values]

In [153]:
query_gastos = '''
INSERT INTO gastos(anio,ncodi,totalcompra,producfarma,materialsani,implantes,restomateriasani,servcontratado,trabajocontratado,xrestocompras,variaexistencias,servexteriores,sumistro,xrestoserviexter,gastopersonal,sueldos,indemnizacion,segsocempresa,otrgassocial,dotaamortizacion,perdidadeterioro,xrestogasto,totcompragasto)
            values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
'''
conexion = conectar(ddbb = 'sanidad42')
query_commit_many(connection = conexion, query_text=query_gastos, valores = lista_tuplas_gastos)


Done!
